In [1]:
#ParallelCellularAlgorithms
import numpy as np
from multiprocessing import Pool

def update_cell(cell_index, grid, size):
    # Unpack the cell index (cell_index should be a tuple (x, y))
    x, y = cell_index

    # Define neighbors' indices (using a 2D Moore neighborhood)
    neighbors = [
        ((x-1) % size, y), ((x+1) % size, y),
        (x, (y-1) % size), (x, (y+1) % size)
    ]

    # Compute new state (this could be any rule like majority, XOR, etc.)
    new_state = sum(grid[n[0], n[1]] for n in neighbors) % 2  # example: majority rule
    return (x, y, new_state)

def parallel_update(grid, size, num_iterations):
    pool = Pool(processes=4)  # Use 4 processes for parallel execution

    for iteration in range(num_iterations):
        print(f"Iteration {iteration + 1}:")

        indices = [(x, y) for x in range(size) for y in range(size)]
        result = pool.starmap(update_cell, [(i, grid, size) for i in indices])

        # Update the grid with the new states
        for x, y, new_state in result:
            grid[x, y] = new_state

        # Print the updated grid for this iteration
        print(grid)

    return grid

# Initialize grid with random states (0 or 1)
grid_size = 5
grid = np.random.randint(2, size=(grid_size, grid_size))

# Number of iterations to run
num_iterations = 10

# Parallel update and print iterations
updated_grid = parallel_update(grid, grid_size, num_iterations)

Iteration 1:
[[1 1 1 0 0]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 0 0 0 1]
 [0 0 1 0 0]]
Iteration 2:
[[0 0 0 0 0]
 [0 1 1 1 1]
 [0 1 1 1 0]
 [1 0 0 1 0]
 [0 0 1 1 1]]
Iteration 3:
[[0 1 0 0 0]
 [0 0 1 1 1]
 [0 0 1 1 0]
 [0 0 1 0 1]
 [0 1 1 1 1]]
Iteration 4:
[[1 1 1 0 0]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 0 0 0 1]
 [0 0 1 0 0]]
Iteration 5:
[[0 0 0 0 0]
 [0 1 1 1 1]
 [0 1 1 1 0]
 [1 0 0 1 0]
 [0 0 1 1 1]]
Iteration 6:
[[0 1 0 0 0]
 [0 0 1 1 1]
 [0 0 1 1 0]
 [0 0 1 0 1]
 [0 1 1 1 1]]
Iteration 7:
[[1 1 1 0 0]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 0 0 0 1]
 [0 0 1 0 0]]
Iteration 8:
[[0 0 0 0 0]
 [0 1 1 1 1]
 [0 1 1 1 0]
 [1 0 0 1 0]
 [0 0 1 1 1]]
Iteration 9:
[[0 1 0 0 0]
 [0 0 1 1 1]
 [0 0 1 1 0]
 [0 0 1 0 1]
 [0 1 1 1 1]]
Iteration 10:
[[1 1 1 0 0]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 0 0 0 1]
 [0 0 1 0 0]]


In [5]:
import random
import operator
import math

# Define operators for expressions
OPERATORS = ['+', '-', '*', '/']
TERMINALS = ['x', '1', '2', '3']

# Function to generate random chromosome
def generate_chromosome(length=10):
    chromosome = []
    for i in range(length):
        if i % 2 == 0:  # Ensure operands are placed at even indices
            chromosome.append(random.choice(TERMINALS))
        else:  # Ensure operators are placed at odd indices
            chromosome.append(random.choice(OPERATORS))
    return chromosome

# Decode chromosome to expression string
def decode_chromosome(chromosome):
    return " ".join(chromosome)

# Evaluate expression with error handling
def evaluate_expression(expr, x_value):
    try:
        # Replace 'x' with a specific value
        expr_with_value = expr.replace('x', str(x_value))
        return eval(expr_with_value)
    except (ZeroDivisionError, SyntaxError):  # Catch syntax errors too
        return float('inf')

# Fitness function: Mean Squared Error
def fitness(chromosome, data):
    expression = decode_chromosome(chromosome)
    error = 0
    for x, y_true in data:
        y_pred = evaluate_expression(expression, x)
        if y_pred == float('inf'):
            return float('inf')
        error += (y_true - y_pred) ** 2
    return error / len(data)

# Generate initial population
def generate_population(size, length=10):
    return [generate_chromosome(length) for _ in range(size)]

# Selection: Tournament selection
def select(population, fitnesses):
    tournament = random.sample(list(zip(population, fitnesses)), 3)
    tournament.sort(key=lambda x: x[1])
    return tournament[0][0]

# Mutation operator
def mutate(chromosome):
    idx = random.randint(0, len(chromosome) - 1)
    if idx % 2 == 0:  # Mutate operands
        chromosome[idx] = random.choice(TERMINALS)
    else:  # Mutate operators
        chromosome[idx] = random.choice(OPERATORS)

# Main GEP function
def gep(data, generations=50, pop_size=20, chromosome_length=10):
    population = generate_population(pop_size, chromosome_length)
    for generation in range(generations):
        fitnesses = [fitness(chromosome, data) for chromosome in population]
        # Print best fitness of generation
        best_fit = min(fitnesses)
        best_chromosome = population[fitnesses.index(best_fit)]
        print(f"Generation {generation}: Best Fitness = {best_fit:.4f}")
        print(f"Best Chromosome: {decode_chromosome(best_chromosome)}")

        # Selection and reproduction
        new_population = []
        for _ in range(pop_size):
            parent = select(population, fitnesses)
            offspring = parent[:]
            mutate(offspring)
            new_population.append(offspring)
        population = new_population

    # Return best solution
    fitnesses = [fitness(chromosome, data) for chromosome in population]
    best_fit = min(fitnesses)
    best_chromosome = population[fitnesses.index(best_fit)]
    return decode_chromosome(best_chromosome)

# Example dataset: y = x^2
data = [(x, x**2) for x in range(-5, 6)]
best_solution = gep(data)
print("Best Solution Found:", best_solution)

Generation 0: Best Fitness = inf
Best Chromosome: 3 + 3 / 3 - 2 - 3 -
Generation 1: Best Fitness = inf
Best Chromosome: 2 * 3 / x + x * x /
Generation 2: Best Fitness = inf
Best Chromosome: 2 * 1 / x + x * x /
Generation 3: Best Fitness = inf
Best Chromosome: x * x / x + x * 2 /
Generation 4: Best Fitness = inf
Best Chromosome: 2 * 1 + x + x - x /
Generation 5: Best Fitness = inf
Best Chromosome: 2 + 2 - 2 / 3 * 1 /
Generation 6: Best Fitness = inf
Best Chromosome: 2 / 2 + x + x - x /
Generation 7: Best Fitness = inf
Best Chromosome: 2 / 2 + 2 + x - x /
Generation 8: Best Fitness = inf
Best Chromosome: 2 * 2 + x + x - x /
Generation 9: Best Fitness = inf
Best Chromosome: x + x - 2 * 3 * x *
Generation 10: Best Fitness = inf
Best Chromosome: 1 - 3 / x * 3 * 2 +
Generation 11: Best Fitness = inf
Best Chromosome: 3 + x / x * 3 + x /
Generation 12: Best Fitness = inf
Best Chromosome: 3 * 2 - x + x * x /
Generation 13: Best Fitness = inf
Best Chromosome: 1 - 3 / 2 * 3 + 3 /
Generation 14: B